In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#Feature tools for "Deep Feature Synthesis"
import featuretools as ft

import pyodbc

In [2]:
data = ft.demo.load_mock_customer()

In this toy dataset, there are 3 tables. Each table is called an `entity` in Featuretools.

customers: unique customers who had sessions

sessions: unique sessions and associated attributes

transactions: list of events in this session

In [3]:
customers_df = data["customers"]

In [4]:
customers_df

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [5]:
sessions_df = data["sessions"]

In [6]:
sessions_df.sample(5)

,session_id,customer_id,device,session_start
13,14,1,tablet,2014-01-01 03:28:00
6,7,3,tablet,2014-01-01 01:39:40
1,2,5,mobile,2014-01-01 00:17:20
28,29,1,mobile,2014-01-01 07:10:05
24,25,3,desktop,2014-01-01 05:59:40


In [7]:
transactions_df = data["transactions"]

In [8]:
transactions_df.sample(5)

,transaction_id,session_id,transaction_time,product_id,amount
74,232,5,2014-01-01 01:20:10,1,139.20
231,27,17,2014-01-01 04:10:15,2,90.79
434,36,31,2014-01-01 07:50:10,3,62.35
420,56,30,2014-01-01 07:35:00,3,72.70
54,444,4,2014-01-01 00:58:30,4,43.59


First, we specify a dictionary with all the entities in our dataset.

In [9]:
entities = {
            "customers" : (customers_df, "customer_id"),
            "sessions" : (sessions_df, "session_id", "session_start"),
            "transactions" : (transactions_df, "transaction_id", "transaction_time")
            }

Second, we specify how the entities are related. When two entities have a one-to-many relationship, we call the “one” enitity, the “parent entity”. A relationship between a parent and child is defined like this: `(parent_entity, parent_variable, child_entity, child_variable)`

In this dataset we have two relationships

In [10]:
relationships = [("sessions", "session_id", "transactions", "session_id"),
                ("customers", "customer_id", "sessions", "customer_id")]


### Run Deep Feature Synthesis
A minimal input to DFS is a set of entities, a list of relationships, and the “target_entity” to calculate features for. The ouput of DFS is a feature matrix and the corresponding list of feature definitions.

Let’s first create a feature matrix for each customer in the data

In [11]:
feature_matrix_customers, features_defs_customers = ft.dfs(entities=entities,
                                                    relationships=relationships,
                                                    target_entity="customers")

In [12]:
feature_matrix_customers

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.product_id),MODE(transactions.product_id),DAY(date_of_birth),DAY(join_date),YEAR(date_of_birth),YEAR(join_date),MONTH(date_of_birth),MONTH(join_date),WEEKDAY(date_of_birth),WEEKDAY(join_date),SUM(sessions.MEAN(transactions.amount)),SUM(sessions.SKEW(transactions.amount)),SUM(sessions.STD(transactions.amount)),SUM(sessions.NUM_UNIQUE(transactions.product_id)),SUM(sessions.MAX(transactions.amount)),SUM(sessions.MIN(transactions.amount)),STD(sessions.MEAN(transactions.amount)),STD(sessions.SKEW(transactions.amount)),STD(sessions.SUM(transactions.amount)),STD(sessions.NUM_UNIQUE(transactions.product_id)),STD(sessions.MAX(transactions.amount)),STD(sessions.COUNT(transactions)),STD(sessions.MIN(transactions.amount)),MAX(sessions.MEAN(transactions.amount)),MAX(sessions.SKEW(transactions.amount)),MAX(sessions.SUM(transactions.amount)),MAX(sessions.STD(transactions.amount)),MAX(sessions.NUM_UNIQUE(transactions.product_id)),MAX(sessions.COUNT(transactions)),MAX(sessions.MIN(transactions.amount)),SKEW(sessions.MEAN(transactions.amount)),SKEW(sessions.SUM(transactions.amount)),SKEW(sessions.STD(transactions.amount)),SKEW(sessions.NUM_UNIQUE(transactions.product_id)),SKEW(sessions.MAX(transactions.amount)),SKEW(sessions.COUNT(transactions)),SKEW(sessions.MIN(transactions.amount)),MIN(sessions.MEAN(transactions.amount)),MIN(sessions.SKEW(transactions.amount)),MIN(sessions.SUM(transactions.amount)),MIN(sessions.STD(transactions.amount)),MIN(sessions.NUM_UNIQUE(transactions.product_id)),MIN(sessions.MAX(transactions.amount)),MIN(sessions.COUNT(transactions)),MEAN(sessions.MEAN(transactions.amount)),MEAN(sessions.SKEW(transactions.amount)),MEAN(sessions.SUM(transactions.amount)),MEAN(sessions.STD(transactions.amount)),MEAN(sessions.NUM_UNIQUE(transactions.product_id)),MEAN(sessions.MAX(transactions.amount)),MEAN(sessions.COUNT(transactions)),MEAN(sessions.MIN(transactions.amount)),NUM_UNIQUE(sessions.MONTH(session_start)),NUM_UNIQUE(sessions.MODE(transactions.product_id)),NUM_UNIQUE(sessions.WEEKDAY(session_start)),NUM_UNIQUE(sessions.DAY(session_start)),NUM_UNIQUE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.WEEKDAY(session_start)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),NUM_UNIQUE(transactions.sessions.customer_id),NUM_UNIQUE(transactions.sessions.device),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device)
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,126,5,4,18,17,1994,2011,7,4,0,6,582.193117,-0.476122,312.745952,40,1057.97,78.59,13.759314,0.589386,279.510713,0.000000,7.322191,4.062019,6.954507,88.755625,0.640252,1613.93,46.905665,5,25,26.36,-0.424949,0.778170,-0.312355,0.000000,-0.780493,1.946018,2.440005,50.623125,-1.038434,809.97,30.450261,5,118.90,12,72.774140,-0.059515,1128.202500,39.093244,5.000000,132.246250,15.750000,9.823750,1,4,1,1,1,1,4,2,1,2014,1,3,1,mobile
2,13244,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,93,5,4,18,15,1986,2012,8,4,0,6,548.905851,-0.277640,258.700528,35,931.63,154.60,11.477071,0.509798,251.609234,0.000000,17.221593,3.450328,15.874374,96.581000,0.755711,1320.64,47.935920,5,18,56.46,0.235296,-0.440929,0.013087,0.000000,-1.539467,-0.303276,2.154929,61.910000,-0.763603,634.84,27.839228,5,100.04,8,78.415122,-0.039663,1028.611429,36.957218,5.000000,133.090000,13.285714,22.085714,1,4,1,1,1,1,3,2,1,2014,1,3,2,desktop
3,13244,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,93,5,1,21,13,2003,2011,11,8,4,5,405.237462,2.286086,257.299895,29,847.63,66.21,11.174282,0.429374,219.021420,0.4

We've made 77 features from the initial 4+4+5 = 14 columns

In [13]:
print(customers_df.shape, sessions_df.shape, transactions_df.shape, feature_matrix_customers.shape)

(5, 4) (35, 4) (500, 5) (5, 77)


### Change target entity
One of the reasons DFS is so powerful is that it can create a feature matrix for any entity in our data. For example, if we wanted to build features for sessions.

In [15]:
feature_matrix_sessions, features_defs_sessions = ft.dfs(entities=entities,
                                                        relationships=relationships,
                                                        target_entity="customers")

In [16]:
feature_matrix_sessions.head()

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.product_id),MODE(transactions.product_id),DAY(date_of_birth),DAY(join_date),YEAR(date_of_birth),YEAR(join_date),MONTH(date_of_birth),MONTH(join_date),WEEKDAY(date_of_birth),WEEKDAY(join_date),SUM(sessions.MEAN(transactions.amount)),SUM(sessions.SKEW(transactions.amount)),SUM(sessions.STD(transactions.amount)),SUM(sessions.NUM_UNIQUE(transactions.product_id)),SUM(sessions.MAX(transactions.amount)),SUM(sessions.MIN(transactions.amount)),STD(sessions.MEAN(transactions.amount)),STD(sessions.SKEW(transactions.amount)),STD(sessions.SUM(transactions.amount)),STD(sessions.NUM_UNIQUE(transactions.product_id)),STD(sessions.MAX(transactions.amount)),STD(sessions.COUNT(transactions)),STD(sessions.MIN(transactions.amount)),MAX(sessions.MEAN(transactions.amount)),MAX(sessions.SKEW(transactions.amount)),MAX(sessions.SUM(transactions.amount)),MAX(sessions.STD(transactions.amount)),MAX(sessions.NUM_UNIQUE(transactions.product_id)),MAX(sessions.COUNT(transactions)),MAX(sessions.MIN(transactions.amount)),SKEW(sessions.MEAN(transactions.amount)),SKEW(sessions.SUM(transactions.amount)),SKEW(sessions.STD(transactions.amount)),SKEW(sessions.NUM_UNIQUE(transactions.product_id)),SKEW(sessions.MAX(transactions.amount)),SKEW(sessions.COUNT(transactions)),SKEW(sessions.MIN(transactions.amount)),MIN(sessions.MEAN(transactions.amount)),MIN(sessions.SKEW(transactions.amount)),MIN(sessions.SUM(transactions.amount)),MIN(sessions.STD(transactions.amount)),MIN(sessions.NUM_UNIQUE(transactions.product_id)),MIN(sessions.MAX(transactions.amount)),MIN(sessions.COUNT(transactions)),MEAN(sessions.MEAN(transactions.amount)),MEAN(sessions.SKEW(transactions.amount)),MEAN(sessions.SUM(transactions.amount)),MEAN(sessions.STD(transactions.amount)),MEAN(sessions.NUM_UNIQUE(transactions.product_id)),MEAN(sessions.MAX(transactions.amount)),MEAN(sessions.COUNT(transactions)),MEAN(sessions.MIN(transactions.amount)),NUM_UNIQUE(sessions.MONTH(session_start)),NUM_UNIQUE(sessions.MODE(transactions.product_id)),NUM_UNIQUE(sessions.WEEKDAY(session_start)),NUM_UNIQUE(sessions.DAY(session_start)),NUM_UNIQUE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.WEEKDAY(session_start)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),NUM_UNIQUE(transactions.sessions.customer_id),NUM_UNIQUE(transactions.sessions.device),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device)
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,126,5,4,18,17,1994,2011,7,4,0,6,582.193117,-0.476122,312.745952,40,1057.97,78.59,13.759314,0.589386,279.510713,0.000000,7.322191,4.062019,6.954507,88.755625,0.640252,1613.93,46.905665,5,25,26.36,-0.424949,0.778170,-0.312355,0.000000,-0.780493,1.946018,2.440005,50.623125,-1.038434,809.97,30.450261,5,118.90,12,72.774140,-0.059515,1128.202500,39.093244,5.000000,132.246250,15.750000,9.823750,1,4,1,1,1,1,4,2,1,2014,1,3,1,mobile
2,13244,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,93,5,4,18,15,1986,2012,8,4,0,6,548.905851,-0.277640,258.700528,35,931.63,154.60,11.477071,0.509798,251.609234,0.000000,17.221593,3.450328,15.874374,96.581000,0.755711,1320.64,47.935920,5,18,56.46,0.235296,-0.440929,0.013087,0.000000,-1.539467,-0.303276,2.154929,61.910000,-0.763603,634.84,27.839228,5,100.04,8,78.415122,-0.039663,1028.611429,36.957218,5.000000,133.090000,13.285714,22.085714,1,4,1,1,1,1,3,2,1,2014,1,3,2,desktop
3,13244,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,93,5,1,21,13,2003,2011,11,8,4,5,405.237462,2.286086,257.299895,29,847.63,66.21,11.174282,0.429374,219.021420,0.4